<h1><center>Projet de Python pour la Data Science<br /><br />Julien Guibert - Charles Rollet - Tanguy Simeon</center></h1>

<img src="LOGO-ENSAE.jpg" style=width:200px;height:400px/>

#### Bienvenue sur le Github de notre projet Python Pour le Data Scientist ! 

L'objectif du projet est de prédire le prix d'un bien immobilier en fonction de certaines de ses caractéristiques telles que sa surface, son nombre de pièces, sa date de vente, son type (appartement ou maison) et sa location à Paris. 

#### Notre projet ce divise en 6 étapes :

<img src="PlanPDS.png" style=width:936px;height:400px/>

#### Veuillez installer au préalable les packages non conventionnels suivant pour utiliser le reste du programme : 

In [4]:
!pip install customtkinter
!pip install tkcalendar
!pip install tkintermapview

SyntaxError: invalid syntax (3787631284.py, line 1)

# Web Scraping

#### Notre démarche :<br />

<div style="text-align: justify">Pour créer un estimateur à partir des caractéristiques de biens déjà vendus, encore faut-il précisément avoir de telles données. Après quelques recherches infructueuses, nous avons souhaité effectuer du Web Scraping grâce au module Selenium et avec l'aide d'un web driver. Technique qui nous a paru essentielle pour obtenir des données sur mesures, nous avons trouvé le site ImmoData (https://www.immo-data.fr/) qui fait l'inventaire des biens vendus en France.<div><br />
    
<div style="text-align: justify">Le site Immodata n'affiche que 100 biens par fenêtre de manière aléatoire, ce qui n'était pas assez et peu précis. Afin de collecter un nombre significatif de données, nous avons enfermé Paris dans un rectangle dont les sommets correspondent à des coordonnées puis diviser ce rectangle en 400 petits rectangles afin de cadriller la ville.<div><br />
    
<div style="text-align: justify">Illustration de notre démarche :<div>

<img src="EncadrementParis.png" style=width:818px;height:200px/>

#### Explication du code :<br />

<div style="text-align: justify">On import d'abord les maodules nécessaire au fonctionnement de l'algorithme et en particulier Selenium<div>



In [ ]:
import argparse
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time
import logging
from tqdm import tqdm
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

<div style="text-align: justify">Veuillez ensuite indiquer le chemin menant à votre webdriver à installer en fonction de votre moteur de recherche et de sa version (nous vous conseillons d'utiliser un chromedriver)<div>

In [ ]:
path = 'Indiquer le chemin'

<div style="text-align: justify">On crée une première fonction afin notamment d'afficher la progression du web scraping et on enregistre la base obtenue sous le nom de 'Base_WS_Brute.csv'<div>

In [ ]:
def main():
    parser = argparse.ArgumentParser() 
    parser.add_argument('--n', type=int, default=2,help='Number of items to search')
    parser.add_argument('--fn', type=str, default='Base_WS_Brute.csv',help='name of the output CSV file')
    args = parser.parse_args()

    start = time.time()
    n = runImmoDataScrapping(n=args.n, filename= args.fn)
    duration = time.time()-start
    logger.info(f'Scrapped {n} items in {duration/60.} minutes. ({str(duration/n).split(".")[0]} seconds per it)')

<div style="text-align: justify">On crée ensuite la fonction runImmoDataScrapping qui va véritablement effectuer le web scraping. Nous avons effectué une double boucle for qui modifie directement la partie du lien url qui concerne les coordonnées géographiques de l'affichage afin d'effectuer l'encadrement de Paris décrit précedemment. Ainsi Paris est contenu dans un rectangle divisé en 440 petits rectangles que nous allons scrapé.<div><br />

<div style="text-align: justify">Une fois les liens à scrapé définis, nous créons un dataframe de cinq colonnes (adresse, type_bien, prix, prix_m2, details qui contient la suface, le nombre de pièces, la date de vente notamment) que nous remplissons avec les informations scrapées correspondantes.<div><br />
    
<div style="text-align: justify">Pour vérifier le bon fonctionnement de l'algorithme, nous vous conseillons de modifier les informations dans le range des deux boucles for en indiquand 256000 à la place de 420030 dans la première et 806001 à la place de 900050 dans la seconde. De cette façon, l'algorithme va scrapé 2x2=4 liens ce qui permettra de gagner du temps.<div>

In [ ]:
def runImmoDataScrapping(n: int, filename: str):
    for i in range(250000,420030,8501):
        for j in range(800001,900050,5001):
            step_x=2
            step_y=48
            driver = webdriver.Chrome(path)
            url = f"https://www.immo-data.fr/explorateur/transaction/recherche?minprice=0&maxprice=5000000&minpricesquaremeter=0&maxpricesquaremeter=40000&propertytypes=1%2C2&minmonthyear=Janvier%202014&maxmonthyear=Juin%202022&nbrooms=1%2C2%2C3%2C4%2C5&minsurface=0&maxsurface=400&minsurfaceland=0&maxsurfaceland=50000&center={step_x+i*0.000001}%3B{step_y+j*0.000001}&zoom=14.5"
        
            driver.get(url)
            driver.implicitly_wait(6)
            colsInDetailsList = ['addresse','type_bien', 'prix', 'prix_m2', 'details']
            data = {colName : [] for colName in colsInDetailsList}
        
            elements = driver.find_elements(by=By.XPATH, value='//div[@class="flex flex-col gap-2 p-3"]')

            for element in tqdm(elements):
                addresse, prix, type_bien, prix_m2, *details = element.text.split('\n')
                
                data['addresse'].append(addresse)
                data['prix'].append(prix)
                data['type_bien'].append(type_bien)
                data['prix_m2'].append(prix_m2)
                data['details'].append(details)
        
            driver.close()
            if i==250000 and j==800001 : 
                df = pd.DataFrame(data) 
            else : 
                df=pd.concat([df,pd.DataFrame(data)],ignore_index=True)      

    df.to_csv(filename)
    return df.shape[0]

if __name__=='__main__':

    main()

Nous avons ainsi réussi à obtenir envrion 40.000 données qu'il nous a ensuite fallu traiter. La base issue du web scraping et intitutée Base_WS_Brute est disponible dans le dossier 'Web Scraping' sur GitHub. 

# Traitement de données 

#### Notre démarche : <br />

<div style="text-align: justify">Si les données récoltées à la suite du web scraping contenaient les informations que nous voulions, celles-ci n'étaient pas optimalement disponibles. En effet, la colonne "détails" par exemple comprenait à la fois le nombre de pièces ainsi, la surface ou encore la date de vente. Or nous souhaitions en faire trois colonnes distinctes afin d'utiliser ces variables individuellement. <div><br />
    
<div style="text-align: justify">C'est pourquoi nous nous sommes attelés à traiter notre base de données avec Pandas. En plus des données obtenues, nous souhaitions aussi en ajouter d'autres et en particulier celles en lien avec les coordonnées géographiques (longitude et latitude) grâce à geopandas des biens afin de les utiliser dans le cadre de notre modèle. <div>

#### Explication du code :<br />  

In [ ]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim 
import geopandas
from shapely.geometry import Point

<div style="text-align: justify">On lit la base issue du web scraping ('Base_WS_Brute') en renommant de la même manière les colonnes que celles issues de la base du gouvernement.<div>



In [ ]:
df = pd.read_csv("Insérer la base issue du Web Scraping",\
                 names=['adresse_du_bien', 'valeur_fonciere','type_local', 'prix_m2', 'details'],\
                  index_col=0)
indexNames = df[ df['adresse_du_bien'] == 'addresse'].index
df.drop(indexNames , inplace=True)
df.head()

<div style="text-align: justify">On supprime les doublons possibles issus du web scraping.<div>



In [ ]:
df.drop_duplicates(keep = 'first', inplace=True)

<div style="text-align: justify">Nous avons ensuite créé trois fonctions permettant soit de supprimer des caractères précis ou d'indiquer les endroits où effectuer les séparations des colonnes.<div>

In [ ]:
def func(x):
    if x[0]=='s':
        a=x[1:]
        return str(a)
    else:
         return x

def separator(c):
    return lambda x: pd.Series(str(x).split(c))

def maxi_separator(c, d):
    return lambda x: pd.Series(str(x).split(c, d))

<div style="text-align: justify">On s'occupe d'abord de nettoyer les colonnes valeur_fonciere et prix_m2 en supprimant les chaînes de caractères inutiles.<div>



In [ ]:
df['prix_m2']=df['prix_m2'].str.split('€/m²').str[0]
df['prix_m2']=df['prix_m2'].str.replace(' ', '')
df['valeur_fonciere']=df['valeur_fonciere'].str.split('€').str[0] 
df['valeur_fonciere']=df['valeur_fonciere'].str.replace(' ', '')

<div style="text-align: justify">On supprime ensuite tous les biens qui ne sont pas localisés à Paris et qui viennent du fait que nous avons encadrer Paris qui est de forme ovale dans un rectangle. Ce sont donc les biens de 'coins'.Pour cela, on effectue un test sur la colonne adresse qui consiste à supprimer toutes les lignes dont les cinq derniers caractères sont différents de 'PARIS'.<div>



In [ ]:
indexNames = df[df['ville'] != 'PARIS'].index
df.drop(indexNames , inplace=True)

<div style="text-align: justify">On nettoye la colonne details pour contruire les colonnes nombre_pieces_principales, surface_reelle_bati et date_mutation.<div>



In [ ]:
df[['nombre_pieces_principales','poubelle']] = df.details.apply(maxi_separator('Surface', 1))
df[['surface_reelle_bati','poubelle']] = df.poubelle.apply(maxi_separator('m²', 1))
df[['poubelle','date_mutation']] = df.poubelle.apply(maxi_separator('le', 1))
df[['poubelle','nombre_pieces_principales']] = df.nombre_pieces_principales.apply(maxi_separator('Pièces', 1))
df[['date_mutation','poubelle']] = df.date_mutation.apply(maxi_separator('+', 1))
df.drop(['poubelle', 'details'], axis=1, inplace=True)
liste_col = ['nombre_pieces_principales', 'surface_reelle_bati', 'date_mutation']
for col in liste_col:
    df[col]=df[col].str.split(',').str[1]
for col in liste_col:
    df[col]=df[col].str.split("'").str[1]
df.head()

<div style="text-align: justify">On convertit les chaines de caratères numériques en nombres entiers pour les colonnes appropriées.<div>


In [ ]:
list_newcolumn = ['valeur_fonciere','prix_m2','nombre_pieces_principales','surface_reelle_bati']
for column in list_newcolumn:
    df[column]=df[column].astype(int)

<div style="text-align: justify">On supprime les données qui semblent de mauvaises qualités par rapport au prix moyen du m2 à Paris.<div>

In [ ]:
indexNames = df[(df['prix_m2']< 8000)].index
df.drop(indexNames , inplace=True)
indexNames1 = df[(df['prix_m2']> 19000)].index
df.drop(indexNames1 , inplace=True)

<div style="text-align: justify">Après avoir terminé de traiter les données issues du web scraping, nous avons souhaité en créer de nouvelles en convertissant les adresses précises des biens en coordonnées géographiques. Pour cela, nous avons utilisé les modules geopandas et geopy. Nous avons d'abord créé une colonne dans laquelle se trouve les coordonnées des biens avant de supprimer les lignes où ces coordonnées étaient introuvables pour enfin diviser la colonne coordonnée en deux colonnes distinctes contenant les données de longitude et de latitude.<div> 

In [ ]:
geolocator = Nominatim(timeout=5, user_agent = "myGeolocator")
df['coordinates'] = df['adresse_du_bien'].apply(geolocator.geocode)
df = df.dropna() #enlève les lignes avec des 'Nan'
df.reset_index(drop=True) #censer renuméroter le DataFrame mais ça marche pas mdrr
df['longitude'] = df['coordinates'].apply(lambda x: x.point.longitude)
df['latitude'] = df['coordinates'].apply(lambda x: x.point.latitude)

<div style="text-align: justify">On supprime les colonnes qui ne sont plus utiles et réordonne celles utiles afin que la base soit similaire à celle du gouv nettoyée.<div>



In [ ]:
df.drop(['coordinates', 'adresse_du_bien', 'adresse', 'ville', 'prix_m2'], axis=1, inplace=True)
df = df[['date_mutation','valeur_fonciere','type_local','surface_reelle_bati','nombre_pieces_principales','longitude','latitude']]

In [ ]:
df

<div style="text-align: justify">On peut sauvegarder la base qui est disponible sur GitHub dans le dossier 'Traitement de la base de données issue du WS' et intitulée 'Base_WS_traitee'.<div>



In [ ]:
df.to_csv('Base_WS_traitee.csv')

# Statistiques descriptives

# Preprocessing

#### Notre démarche : <br />

<div style="text-align: justify">Notre base comporte à la fois des varibales catégorielles ainsi que des variables continues. Nous avons donc dû traiter les deux types de variables séparément.<div><br />

<div style="text-align: justify">Les variables catégorielles(la date, le nombre de pièces et le type du bien).<div><br /> 

<div style="text-align: justify">Les variables continues (la surface, la longitude et la latitude).<div><br />

<div style="text-align: justify">Nous avons alors créé une fonction nous permettant de choisir le meilleur standardisateur parmis StandardScaler, MinMaxScaler, RobustScaler pour 6 modèles utilisés. Le standardisateur qui avait globalement les meilleurs performances a été ()..<div> 

#### Explication du code :<br />  

<div style="text-align: justify">On importe d'abord plusieurs modules dont les modèles à tester et les standardisateur (LabelEncoder, StandardScaler, RobustScaler,MinMaxScaler) en lien avec sklearn.<div> 


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, RobustScaler,MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.model_selection import cross_val_score

In [ ]:
df_temp = pd.read_csv('Insérer la base Base_WS_traitee')

column = ["date_mutation","valeur_fonciere", "type_local","surface_reelle_bati", "nombre_pieces_principales","longitude", "latitude"]
df = df_temp[column]

#### <div style="text-align: justify">On traite d'abord les variables catégorielles.<div> 



<div style="text-align: justify">On recode en binaire la variable type de bien.<div> 



In [ ]:
encoder = LabelEncoder()
df["type_local"]=encoder.fit_transform(df["type_local"])

<div style="text-align: justify">On regroupe les biens de plus de 9 pièces afin d'éviter le bruit que ces varibales pourraient engendrer.<div>



In [ ]:
conditions = [df['nombre_pieces_principales']<9, df['nombre_pieces_principales']>=9]
values = [df['nombre_pieces_principales'].values, 'more_than_9']
df['test_nb_pieces'] = np.select(conditions, values)

<div style="text-align: justify">Pour traiter la date nous avons converti la colonne en datetime puis créé deux nouvelles colonnes contenant l'année de vente et le mois. Nous avons en effet estimé que le jour de vente n'aurait pas réellement d'impact sur l'estimation du prix. Puis nous avons regroupé les mois en demi-semestres pour gagner en pertinance.
<div> 

In [ ]:
df["date_mutation"]=pd.to_datetime(df["date_mutation"]) 
df['année'] = df['date_mutation'].dt.year
df['mois'] = df['date_mutation'].dt.month
df.sort_values('date_mutation', ignore_index=True)

conditions = [df['mois']<4, (df['mois']>=4) & (df['mois']<7), (df['mois']>=7) & (df['mois']<10),df['mois']>=10]
choiceliste = ['semestre_1','semestre_2','semestre_3','semestre_4']
df['semestre'] = np.select(conditions, choiceliste)


<div style="text-align: justify">Nous avons ensuite encoder ces variables grâce à get_dummies() qui les a converties en indicatrices.



In [ ]:
df_test = pd.get_dummies(df, columns = ['semestre'])
df_final = pd.get_dummies(df_test, columns = ['test_nb_pieces'])

<div style="text-align: justify">On utilise 3 principaux Standardisateurs.<div> 

In [ ]:
sc = StandardScaler()
rs = RobustScaler()
minmax = MinMaxScaler()

<div style="text-align: justify">On standardise les variables continues dans les bases train et test selon les 3 standardisateurs choisis.<div> 

In [ ]:
scale = sc.fit_transform(df_continu)
df_scale = pd.DataFrame(scale, columns = liste_annexe)
scale_test = sc.transform(df_continu_test)
df_scale_test = pd.DataFrame(scale_test, columns = liste_annexe)

robust = rs.fit_transform(df_continu)
df_robust = pd.DataFrame(robust, columns = liste_annexe)
robust_test = rs.transform(df_continu_test)
df_robust_test = pd.DataFrame(robust_test, columns = liste_annexe)

MinMax = minmax.fit_transform(df_continu)
df_MinMax = pd.DataFrame(MinMax, columns = liste_annexe)
MinMax_test = minmax.transform(df_continu_test)
df_MinMax_test = pd.DataFrame(MinMax_test, columns = liste_annexe)

<div style="text-align: justify">On termine de construire nos bases finales standardisées selon les trois estimateurs.<div> 

In [ ]:
X_train_st = X_train.copy()
X_train_rob = X_train.copy()
X_train_mm = X_train.copy()

X_test_st = X_test.copy()
X_test_rob = X_test.copy()
X_test_mm = X_test.copy()

for col in liste_annexe:
    X_train_st[col] = df_scale[col]
    X_test_st[col] = df_scale_test[col]
    
for col in liste_annexe:
    X_train_rob[col] = df_robust[col]
    X_test_rob[col] = df_robust_test[col]
    
for col in liste_annexe:
    X_train_mm[col] = df_MinMax[col]
    X_test_mm[col] = df_MinMax_test[col]

<div style="text-align: justify">On crée une fonction pour voir quel est le meilleur standardisateur pour chaque modèle choisi.<div> 

In [ ]:
def meilleur_standardiseur(model_name):
    
    better_score = 0
    better_scaler = ''

    X_train_list = [X_train_st, X_train_rob, X_train_mm]
    X_test_list = [X_test_st, X_test_rob, X_test_mm]
    scalers = ["Standard", "Robust", "MinMax"]
    
    for i in range(len(X_test_list)):
        model = model_name
        model.fit(X_train_list[i], Y_train)
        score = model.score(X_test_list[i], Y_test)
        if score > better_score:
            better_score = score
            better_scaler = scalers[i]
        elif score == better_score:
            better_scaler += " / " + scalers[i]
    print("Pour le modèle {}, better score {} avec {}.".format(model_name, better_score, better_scaler))

<div style="text-align: justify">On applique la fonction à 6 modèles différents.<div> 

In [ ]:
meilleur_standardiseur(LinearRegression())

In [ ]:
meilleur_standardiseur(RandomForestClassifier())

In [ ]:
meilleur_standardiseur(DecisionTreeClassifier())

In [ ]:
meilleur_standardiseur(LogisticRegression())

In [ ]:
meilleur_standardiseur(KNeighborsClassifier())

In [ ]:
meilleur_standardiseur(SVC())

#### <div style="text-align: justify"> Résultat du meilleur standardisateur selon les modèles utilisés :<div> 

|          | Decision Tree Method | Random Forest Method | K Neighbors Classifier | Support Vector Machine | Logistic Regression | Linear Regression | 
|:--------:| :------------------: | :------------------: | :--------------------: | :--------------------: | :-----------------: | :---------------: |
| Robust   |          ✅          |          ?           |           ✅           |          ?             |         ❌          |        ✅        |
| Standard |          ❌          |          ?           |           ❌           |          ?             |         ❌          |        ❌        |
| MinMax   |          ❌          |          ?           |           ❌           |          ?             |         ✅          |        ❌        |


<div style="text-align: justify">On peut enregistrer les bases train et test standardisés selon le meilleur standardisateur de chaque modèle.<div> 

In [ ]:
df_new.to_csv('base finale pour le ML')

# Modélisation 

#### Notre démarche : <br />

<div style="text-align: justify">Après avoir choisi le meilleur standardisateur pour chaque modèle utiliser, nous allons créer une fonction permettant de déterminer les hyperparamètres optimaux de chaque modèle parmi ceux sélectionnés.<div><br />

#### Explication du code :<br />  

In [ ]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.pipeline import make_pipeline
from statistics import mean

<div style="text-align: justify">On définit de dictionnaires concernant les hyperparamètres de chaque modèle selectionné.<br />

In [ ]:
dico_param_tree = {'decisiontreeclassifier__criterion': ['gini', 'entropy'],
         'decisiontreeclassifier__splitter': ['best', 'random']}
dico_param_rf = {"randomforestclassifier__n_estimators": np.arange(10, 100, 20),
              "randomforestclassifier__criterion": ['gini', 'entropy'],
              "randomforestclassifier__max_features": ['auto', 'sqrt', 'log2'],
              "randomforestclassifier__class_weight": [None, 'balanced']}
dico_param_log = {'logisticregression__solver': ['newton-cg', 'lbfgs', 
                                                 'liblinear', 'sag', 'saga']}
dico_param_knn = {"kneighborsclassifier__n_neighbors":np.arange(2, 10, 1),
                 "kneighborsclassifier__weights": ['uniform', "distance"],
                 "kneighborsclassifier__algorithm": ["auto", "ball_tree", "kd_tree", "brute"]}
dico_param_svm = {'svc__kernel':["linear", "poly", "rbf", "sigmoid"], 
           "svc__gamma":['scale', "auto"],
                 "svc__probability":[True, False]}

<div style="text-align: justify">On utilise ma
    ke_pipeline pour sauvegarder nos modèles et éviter de relancer des calcules conséquents à chaque fois. <div>

In [ ]:
model_rf = make_pipeline(RandomForestClassifier())
model_tree = make_pipeline(DecisionTreeClassifier())
model_log = make_pipeline(LogisticRegression())
model_knn = make_pipeline(KNeighborsClassifier())
model_svm = make_pipeline(SVC())

<div style="text-align: justify">On crée une fonction qui donne les meilleurs hyperparamètres de chaque modèle. <div>

In [ ]:
# On souhaite que le recall et la precision soient les scores qui comptent le plus
SCORING = 'f1_weighted' 

def best_param(X, Y, pipeline, dico_param):
    
    # On crée un grid search pour obtenir les meilleurs hyperparamètres
    grid = GridSearchCV(
        pipeline, 
        param_grid=dico_param,  
        cv=5, 
        refit=True,
        scoring=SCORING)
    
    grid.fit(X, Y)
    print(f"Le meilleur estimateur a {SCORING}={round(grid.best_score_, 2)}")
    
    # On garde le meilleur modèle en mémoire 
    best_model = grid.best_estimator_
    print("Le meilleur modèle est :", best_model[0])
    return best_model[0]

In [ ]:
<div style="text-align: justify">On crée une fonction qui donne les meilleurs hyperparamètres de chaque modèle. <div>

In [ ]:
best_model_log = best_param(X_train, Y_train, model_log, dico_param_log)

In [ ]:
best_model_tree = best_param(X_train, Y_train, model_tree, dico_param_tree)

In [ ]:
best_model_knn = best_param(X_train, Y_train, model_knn, dico_param_knn)

In [ ]:
best_model_svm = best_param(X_train, Y_train, model_svm, dico_param_svm)

In [ ]:
best_model_rf = best_param(X_train, Y_train, model_rf, dico_param_rf)

<div style="text-align: justify">On crée une fonction pour tester le meilleur modèle et obtenir différents résultats sur ses performances. <div>

In [ ]:
def fit(X_train, X_test, Y_train, model):
    
    accuracy = mean(cross_val_score(model, X_train, Y_train, cv=(), scoring='accuracy'))
    auc = mean(cross_val_score(model, X_train, Y_train, cv=StratifiedKFold(), scoring='roc_auc'))
    precision = mean(cross_val_score(model, X_train, Y_train, cv=StratifiedKFold(), scoring='precision'))
    recall = mean(cross_val_score(model, X_train, Y_train, cv=StratifiedKFold(), scoring='recall'))
    f1_weighted = mean(cross_val_score(model, X_train, Y_train, cv=StratifiedKFold(), scoring='f1_weighted'))
    proba = model.predict_proba(X_test)[:,1]
    
    print('Accuracy: {:.3g}'.format(accuracy))
    print('Precision: {:.3g}'.format(precision))
    print('Recall: {:.3g}'.format(recall))
    print('F1 weighted: {:.3g}'.format(f1_weighted))
    print('AUC score: {:.3g}'.format(auc))
    
    return accuracy, auc, precision, f1_weighted, proba, recall

 <div style="text-align: justify">On applique la fonction au meilleur modèle. <div>

In [ ]:
accuracy_log, auc_log, precision_log, f1_weighted_log, proba_log, recall_log = fit(X_train, X_test, Y_train, best_model_log)

#### <div style="text-align: justify">Résultats des modèles utilisés : <div><br />

|          | Decision Tree Method | Random Forest Method | K Neighbors Classifier | Support Vector Machine | Logistic Regression | Linear Regression | 
|:--------:| :------------------: | :------------------: | :--------------------: | :--------------------: | :-----------------: | :---------------: |
| Score    |          ?           |          ?           |            ?           |          ?             |         ?           |       ?           |
| Accuracy |          ?           |         ?            |          ?             |          ?             |       ?             |     ?             |

Ainsi, nous avons décidé de choisir le modèle XXX avec comme standardisateur XXX et comme hyperparamètres XXX pour estimer le prix d'un bien selon certaines de ses caractéristiques. 

# Visualisation grâce à une interface graphique

#### Notre démarche : <br />

<div style="text-align: justify">Plutôt de rentrer directement les caractéristiques du bien dans Python, nous avons souhaité développer une certaine expérience utilisateur tant pour allier forme et fond que pour créer une interface dynamique. Nous avons aussi voulu utiliser le plus de fonctionalités du module associé à l'interface et c'est pourquoi nous avons varié la nature des widgets utilisés tout en essayant de trouver le plus adapté à l'entrée du paramètre.<div> 

#### Explication du code :<br />  

Pour cela on utilise des modules non convetionnels qu'il faudra installer au préalable tels que tkintermapview, customtkinter ou encore tkcalendar.

In [ ]:
import tkinter
import tkinter.messagebox
import tkintermapview
import customtkinter
import tkcalendar
import geopandas
from geopy.geocoders import Nominatim 
import pandas as pd
import numpy as np

<div style="text-align: justify">Après ce travail préalable, nous avons véritablement créé notre interface et utilisé des widgets différents (Entry, commonbox, button etc.) pour le meilleur rendu possible. Nous avons souhaité créer un compteur pour indiquer le nombre de pièces du bien en liant une fonction + et une fonction - à leur bouton repectif qui modifient le texte d'un Label. <div><br />
    
<div style="text-align: justify">Nous avons aussi introduit un calendrier pour la saisie de la date qui apparait lorsqu'un bouton est actionné et disparait en affichant la date dans un label lorsqu'un autre l'est aussi. Enfin, nous avons relié la fonction estimateur par l'intermédiaire de la fonction callback à un bouton central. <div> 

In [ ]:
global df

customtkinter.set_appearance_mode("System") 
customtkinter.set_default_color_theme("dark-blue") 


class App(customtkinter.CTk):

    WIDTH = 1080
    HEIGHT = 950
    
    def __init__(self):
        super().__init__()

        self.title("Welcome at JCT Company.py")
        self.geometry(f"{App.WIDTH}x{App.HEIGHT}")
        self.protocol("WM_DELETE_WINDOW", self.on_closing) 

        # ======================== Création de deux frame ========================

        # configure grid layout (2x1)
        
        self.grid_columnconfigure(0, weight = 0)
        self.grid_columnconfigure(1, weight = 1)
        self.grid_columnconfigure(2, weight = 0)
        self.grid_rowconfigure(0, weight=0)
        self.grid_rowconfigure(1, weight=1)

        self.frame_left = customtkinter.CTkFrame(master=self,
                                                 width=100,
                                                 corner_radius=0)
        self.frame_left.grid(row=0, column=0, sticky="nswe")
        
        self.frame_right = customtkinter.CTkFrame(master=self,
                                                 width=20,
                                                 corner_radius=0, 
                                                 fg_color = "transparent")
        self.frame_right.grid(row=0, column=2, sticky="nswe")

        
        # ============ Titre général du frame_left ============
        
        self.label_titre = customtkinter.CTkLabel(master=self.frame_left,
                                              text="Paramètres du bien à estimer",
                                              font=("Roboto Medium", -25))
        self.label_titre.grid(row=0, column=0, columnspan=2, pady=20, padx=10, sticky="nswe")
    
    
        # ============ Insertion de l'adresse du bien et de son arrondissement ============

        self.entry_adresse = customtkinter.CTkEntry(master=self.frame_left,
                                            width=100,
                                            placeholder_text="Adresse du bien")     
        self.entry_adresse.grid(row=1, column=0, pady=30, padx=10, sticky="we")
        
        self.combobox_arrondissement = customtkinter.CTkComboBox(master=self.frame_left,
                                                    values=["75001", "75002", "75003", "75004", "75005", "75006", "75007", "75008", "75009", "75010","75011", "75012", "75013", "75014", "75015","75016", "75017", "75018", "75019", "75020"])
        self.combobox_arrondissement.grid(row=1, column=1, pady=30, padx=10, sticky="we")
        self.combobox_arrondissement.set("Arrondissement")
        
        
        # ============ Insertion de la surface du bien ============
        
        self.entry_surface = customtkinter.CTkEntry(master=self.frame_left,
                                            width=100,
                                            placeholder_text="Surface du bien (en m2)")
        self.entry_surface.grid(row=2, column=0, pady=30, padx=10, sticky="we")


        # ============ Précision du type de bien (maison ou appartement) ============

        self.radio_var = tkinter.IntVar(value=0)
        
        self.radio_button_appartement = customtkinter.CTkRadioButton(master=self.frame_left,text="Appartement",
                                                           variable=self.radio_var,
                                                           value=0)
        self.radio_button_appartement.grid(row=3, column=0,pady=30, padx=10, sticky="nswe")

        self.radio_button_maison = customtkinter.CTkRadioButton(master=self.frame_left, text="Maison",
                                                           variable=self.radio_var,
                                                           value=1)
        self.radio_button_maison.grid(row=3, column=0,pady=30, padx=150, sticky="nswe")


        # ============ Insertion du prix d'achat du bien ============

        self.entry_prix = customtkinter.CTkEntry(master=self.frame_left,
                                            width=150,
                                            placeholder_text="Prix d'achat du bien")      
        self.entry_prix.grid(row=6, column=0, pady=30, padx=10, sticky="we")
        
        # ============ Définition d'une pièce ============
        
        self.label_info = customtkinter.CTkLabel(master=self.frame_left,
                                                   text="Indiquez le nombre de pièces \n" + "(On considère comme étant une pièce tout endroit de plus de 9m2 \n" +
                                                   "les salles de bains et WC ne sont pas pris en compte)",
                                                   width=400,
                                                   corner_radius=6,  
                                                   fg_color=("white", "gray38"),
                                                   justify=tkinter.LEFT)
        self.label_info.grid(row=4, column=0, pady=30, padx=10, sticky="w")
        
        
        # ============ Compteur pour indiquer le nombre de pièces ============
        
        self.res_piece = customtkinter.CTkLabel(master=self.frame_left,text='0')
        self.res_piece.grid(row=4, column=1, padx=10, pady=50, sticky="ew")
        
        
        # ============ Création de deux boutons pour régler le nombre de pièces du bien ============
        def compteur_plus():
            a= int(self.res_piece.cget("text"))
            if a >= 50:
                a = a
            else:
                a = a+1
            self.res_piece.configure(text=str(a))
            global nbp #crée une variable globale qu'on peut appeler n'importe où dans le code
            nbp = a
        
        def compteur_moins():
            b = int(self.res_piece.cget("text"))
            if b==0:
                b=b
            else:
                b = b-1
            self.res_piece.configure(text=str(b))
            global nbp #crée une variable globale qu'on peut appeler n'importe où dans le code
            nbp = b

        self.bouton_compteur_plus = customtkinter.CTkButton(master=self.frame_left, text="+", width=26, 
                                                    height=26,
                                                    command=compteur_plus)
        self.bouton_compteur_plus.place(x=521,y=380)
        
        self.bouton_compteur_moins = customtkinter.CTkButton(master=self.frame_left, text="-", width=26, 
                                                    height=26,
                                                    command=compteur_moins)
        self.bouton_compteur_moins.place(x=471,y=380)
        

        # ============ Précision de la date du jour ============
        def calendrier():
            #date_today = ''
            self.cal = tkcalendar.Calendar(self, selectmode = 'day', year = 2022, month = 12, day = 25)
            self.cal.place(x=950, y=490)
            def grad_date():
                #self.label_mask = customtkinter.CTkLabel(self.frame_left, width = 10)
                #self.label_mask.grid(row=11, column=1, pady=10, padx=10)
                self.date = customtkinter.CTkLabel(self, text = "") 
                self.date.configure(text = self.cal.get_date()) 
                self.cal.destroy()
                self.button_date.destroy()
                global date_today
                self.date_today = self.date.cget("text")
                date_today = self.date_today
                #affichage de la date
                self.label_calendrier = customtkinter.CTkLabel(master=self.frame_left,
                                                            text=self.date_today,
                                                            font=("Roboto Medium", -12),
                                                            corner_radius=6)
                self.label_calendrier.place(x=360, y=485)
            
            self.button_date=customtkinter.CTkButton(self, text = "OK", command = grad_date)
            self.button_date.place(x=650, y=485)
        
        self.label_calendrier = customtkinter.CTkLabel(master=self.frame_left,
                                                    text="Veuillez entrez la date du jour",
                                                    font=("Roboto Medium", -16),
                                                    corner_radius=0)
        self.label_calendrier.grid(row=5, column=0, pady=30, padx=0)

        self.botondate=customtkinter.CTkButton(self.frame_left, text='Date du jour', command=calendrier)
        self.botondate.grid(row=5, column=1, pady=30, padx=10)

        # ============ Changement du mode couleur ============

        self.optionmenu = customtkinter.CTkOptionMenu(master=self.frame_left,
                                                        values=["Light", "Dark", "System"],
                                                        command=self.change_appearance)
        self.optionmenu.grid(row=7, column=0, padx=10, pady=30, sticky="ew")

        
        # ======================== frame_right_down ========================
        # Création frame de droite
        
        self.tabview = customtkinter.CTkTabview(self, width=600)
        self.tabview.grid(row=0, column=1, padx=(20, 0), pady=(20, 0), sticky="nsew")
        self.tabview.add("Résultat")
        self.tabview.add("Description")
        self.tabview.tab("Résultat").grid_columnconfigure(0, weight=1)  # configure grid of individual tabs
        self.tabview.tab("Description").grid_columnconfigure(0, weight=1)
        
        
        self.map= tkintermapview.TkinterMapView(self.tabview.tab("Résultat"), width=600, height=450)
        self.map.set_position(48.857345 , 2.347999)
        self.map.set_zoom(12)
        self.map.grid(row=0, column=0, sticky="nswe", padx=(0, 0), pady=(0, 0))
        
        
        self.botonestimation=customtkinter.CTkButton(self.tabview.tab("Résultat"), text='Estimation', command=self.callback)
        self.botonestimation.grid(row=3, column=1, pady=30, padx=0)
        #self.botonestimation.place(x=550, y=550)
        
    # ======================== Espace des fonctions ============
    def test(self):
        self.label_res = customtkinter.CTkLabel(master=self.tabview.tab("Résultat"),
                                                    text="3 mille euros",
                                                    font=("Roboto Medium", -14),
                                                    fg_color = 'transparent',
                                                    corner_radius=6)
        self.label_res.grid(row=1, column=0, pady=0, padx=100)
        
    def button_event(self):
        print("Button pressed")

    def change_appearance(self, new_appearance_mode):
        customtkinter.set_appearance_mode(new_appearance_mode)

    def on_closing(self, event=0):
        self.destroy()
        
    def callback(self):
        #variables
        arr=int(self.combobox_arrondissement.get())
        p=int(self.radio_var.get())
        s=int(self.entry_surface.get())
        k=int(self.res_piece.cget("text"))
        u=int(self.entry_prix.get())
        global date_today
        self.date_today = date_today
        self.adr = str(self.entry_adresse.get())
        self.geolocator = Nominatim(timeout=10, user_agent = "myGeolocator")
        self.adr = self.adr + ' ' + str(arr) + ' ' + 'PARIS'
        self.adr_point = self.geolocator.geocode(self.adr)
        self.lat_bien = self.adr_point.latitude
        self.lgt_bien = self.adr_point.longitude
        
        txt = '     ' + "Valeur estimée du bien :" + '   ' + \
            "3€" + str(p) + str(s) + str(k) + str(u) + str(date_today) + '     '
        #    str(self.res[0]) + ' ' + '€'
        self.label_res = customtkinter.CTkLabel(master=self.tabview.tab("Résultat"),
                                                    text=txt,
                                                    font=("Roboto Medium", -14),
                                                    fg_color = 'transparent',
                                                    corner_radius=6)
        self.label_res.grid(row=1, column=0, pady=30, padx=0)
        
        #plus_value_res = self.res[0] - u
        txt = '     ' + 'Plus value estimée possible :' + '   ' + \
            "3€" + '     '
            #str(plus_value_res) + ' ' + '€'
        self.label_res_pv = customtkinter.CTkLabel(master=self.tabview.tab("Résultat"),
                                                    text=txt,
                                                    font=("Roboto Medium", -14),
                                                    fg_color = 'transparent',
                                                    corner_radius=6)
        self.label_res_pv.grid(row=2, column=0, pady=0, padx=0)
        
        #carte
        self.map= tkintermapview.TkinterMapView(self.tabview.tab("Résultat"), width=600, height=450)
        self.map.set_position(48.857345 , 2.347999)
        self.map.set_zoom(12)
        self.map.grid(row=0, column=0, sticky="nswe", padx=(0, 0), pady=(0, 0))
        self.map.set_marker(self.lat_bien,self.lgt_bien)
       

if __name__ == "__main__":
    app = App()
    app.mainloop()
    

<div style="text-align: justify">La fonction callback sert d'intermédiaire pour récupérer les paramètres à implémenter dans le modèle depuis les widgets du module tkinter.<div><br />
    
<div style="text-align: justify">Pour se faire, nous avons utilisé la commande get en prenant soin de convertir les chaines de caractères en nombres lorsque cela était nécessaire. Nous avons aussi dû appliquer notre algorithme issu de geopandas pour transformer l'adresse entrée en coordonnées, étant des paramètres du modèle.<div><br /> 
    
<div style="text-align: justify">Enfin, nous avons intégré à la fonction callback l'affichage des points des biens utilisés (en bleu) pour estimer le prix du bien voulu (en rouge) sur la carte ainsi que l'appartition du texte donnant le résultat de l'algorithme et quelques détails (nombre de biens comparés, plus value possible) en faisant attention à ce que dès que la fonction callback soit actionnée, les labels soient effacés et non pas surposés les uns sur les autres.<div> 